In [1]:
import findspark

In [2]:
findspark.init('/home/sushant/spark-2.1.0-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('kmeans').getOrCreate()

In [5]:
from pyspark.ml.clustering import KMeans

In [6]:
!pwd

/home/sushant/Documents/SparkUdemyCourse


In [7]:
data = spark.read.format('libsvm').load('./Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Clustering/sample_kmeans_data.txt')

In [8]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|           (3,[],[])|
|  1.0|(3,[0,1,2],[0.1,0...|
|  2.0|(3,[0,1,2],[0.2,0...|
|  3.0|(3,[0,1,2],[9.0,9...|
|  4.0|(3,[0,1,2],[9.1,9...|
|  5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+



Since, we are doing an unsupervised problem here, we don't actually need the labels.

In [9]:
dataUnlabeled = data.select('features')

In [10]:
dataUnlabeled.show(3)

+--------------------+
|            features|
+--------------------+
|           (3,[],[])|
|(3,[0,1,2],[0.1,0...|
|(3,[0,1,2],[0.2,0...|
+--------------------+
only showing top 3 rows



In [11]:
kmeans = KMeans().setK(2).setSeed(1)
# seed is for the random number generator which chooses the initial centroids. Set the seed for repeatability. 

In [13]:
model = kmeans.fit(dataUnlabeled)

In [14]:
withinSetSumOfSquareErrors = model.computeCost(dataUnlabeled)

In [15]:
print(withinSetSumOfSquareErrors)

0.11999999999994547


In [16]:
dataUnlabeled.show()

+--------------------+
|            features|
+--------------------+
|           (3,[],[])|
|(3,[0,1,2],[0.1,0...|
|(3,[0,1,2],[0.2,0...|
|(3,[0,1,2],[9.0,9...|
|(3,[0,1,2],[9.1,9...|
|(3,[0,1,2],[9.2,9...|
+--------------------+



Note that the features are three dimensional. So cluster centers will also be in a three-dimensional space.

In [17]:
model.clusterCenters()

[array([0.1, 0.1, 0.1]), array([9.1, 9.1, 9.1])]

Note that we set K = 2. 

In [18]:
model.transform(dataUnlabeled).show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         0|
|(3,[0,1,2],[0.1,0...|         0|
|(3,[0,1,2],[0.2,0...|         0|
|(3,[0,1,2],[9.0,9...|         1|
|(3,[0,1,2],[9.1,9...|         1|
|(3,[0,1,2],[9.2,9...|         1|
+--------------------+----------+



Seeds data set

In [19]:
data = spark.read.csv('./Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Clustering/seeds_dataset.csv', 
                     header = True, inferSchema=True)

In [20]:
data.show(3)

+-----+---------+-----------+-----------------+------------------+---------------------+----------------+
| area|perimeter|compactness| length_of_kernel|   width_of_kernel|asymmetry_coefficient|length_of_groove|
+-----+---------+-----------+-----------------+------------------+---------------------+----------------+
|15.26|    14.84|      0.871|            5.763|             3.312|                2.221|            5.22|
|14.88|    14.57|     0.8811|5.553999999999999|             3.333|                1.018|           4.956|
|14.29|    14.09|      0.905|            5.291|3.3369999999999997|                2.699|           4.825|
+-----+---------+-----------+-----------------+------------------+---------------------+----------------+
only showing top 3 rows



In [21]:
data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [23]:
data.describe().show()

+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|summary|              area|         perimeter|         compactness|   length_of_kernel|   width_of_kernel|asymmetry_coefficient|   length_of_groove|
+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|  count|               210|               210|                 210|                210|               210|                  210|                210|
|   mean|14.847523809523816|14.559285714285718|  0.8709985714285714|  5.628533333333335| 3.258604761904762|   3.7001999999999997|  5.408071428571429|
| stddev|2.9096994306873647|1.3059587265640225|0.023629416583846364|0.44306347772644983|0.3777144449065867|   1.5035589702547392|0.49148049910240543|
|    min|             10.59|             12.41|              0.8081|              4.899|            

In [24]:
from pyspark.ml.feature import VectorAssembler

In [25]:
data.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [26]:
assembler = VectorAssembler(inputCols=data.columns, outputCol='features')

In [27]:
dataTransformed = assembler.transform(data)

In [28]:
dataTransformed.show(3)

+-----+---------+-----------+-----------------+------------------+---------------------+----------------+--------------------+
| area|perimeter|compactness| length_of_kernel|   width_of_kernel|asymmetry_coefficient|length_of_groove|            features|
+-----+---------+-----------+-----------------+------------------+---------------------+----------------+--------------------+
|15.26|    14.84|      0.871|            5.763|             3.312|                2.221|            5.22|[15.26,14.84,0.87...|
|14.88|    14.57|     0.8811|5.553999999999999|             3.333|                1.018|           4.956|[14.88,14.57,0.88...|
|14.29|    14.09|      0.905|            5.291|3.3369999999999997|                2.699|           4.825|[14.29,14.09,0.90...|
+-----+---------+-----------+-----------------+------------------+---------------------+----------------+--------------------+
only showing top 3 rows



Data scaling

In [29]:
from pyspark.ml.feature import StandardScaler

In [30]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

In [32]:
scaledData = scaler.fit(dataTransformed).transform(dataTransformed)

In [36]:
scaledData.select('features', 'scaledFeatures').show(5)

+--------------------+--------------------+
|            features|      scaledFeatures|
+--------------------+--------------------+
|[15.26,14.84,0.87...|[5.24452795332028...|
|[14.88,14.57,0.88...|[5.11393027165175...|
|[14.29,14.09,0.90...|[4.91116018695588...|
|[13.84,13.94,0.89...|[4.75650503761158...|
|[16.14,14.99,0.90...|[5.54696468981581...|
+--------------------+--------------------+
only showing top 5 rows



In [37]:
scaledData.select('features', 'scaledFeatures').head(5)

[Row(features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621])),
 Row(features=DenseVector([14.88, 14.57, 0.8811, 5.554, 3.333, 1.018, 4.956]), scaledFeatures=DenseVector([5.1139, 11.1566, 37.2883, 12.5354, 8.8241, 0.6771, 10.0838])),
 Row(features=DenseVector([14.29, 14.09, 0.905, 5.291, 3.337, 2.699, 4.825]), scaledFeatures=DenseVector([4.9112, 10.789, 38.2997, 11.9419, 8.8347, 1.7951, 9.8173])),
 Row(features=DenseVector([13.84, 13.94, 0.8955, 5.324, 3.379, 2.259, 4.805]), scaledFeatures=DenseVector([4.7565, 10.6742, 37.8977, 12.0163, 8.9459, 1.5024, 9.7766])),
 Row(features=DenseVector([16.14, 14.99, 0.9034, 5.658, 3.562, 1.355, 5.175]), scaledFeatures=DenseVector([5.547, 11.4782, 38.232, 12.7702, 9.4304, 0.9012, 10.5294]))]

In [38]:
kmeans = KMeans(featuresCol='scaledFeatures', k=3)